In [4]:
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model


from env_test import GridEnvironment
from DQL_agent import DQNAgent

In [5]:
# Load data
N = 10 # 10 EVs

# Get demand data
# TODO get apparent power from real and reactive, use just real for now
with open('Building Load Data/real_power_data.json', 'r') as json_file:
    real_dict = json.load(json_file)
    for key in real_dict.keys(): # get data from first key only (CAPTL_WF)
        demand_data = np.array(real_dict[key])
        break

# Get solar data
with open('PV Generation Data/pv_data.json', 'r') as json_file:
    pv_dict = json.load(json_file)
    for key in pv_dict.keys(): # get data from first key only (CAPTL_WF)
        solar_data = np.array(pv_dict[key])
        break

# Get wind data
with open('Wind Data/wind_data.json', 'r') as json_file:
    wind_dict = json.load(json_file)
    for key in wind_dict.keys(): # get data from first key only (CAPTL_WF)
        wind_data = np.array(wind_dict[key])
        break

In [6]:
# Initialize environment with your chosen day's data
day_index=0
timestep_length=.25 #in hours
batch_size = 32 #TWEAk
env = GridEnvironment(N, demand_data, solar_data, wind_data, day_index, timestep_length)

# Initialize DQN agent
agent = DQNAgent(state_size=env.state_size, action_size=env.action_size)

# Run for one episode (one day)
state = env.get_state()  # Reset environment to start state
total_reward = 0

T=96 #(15 minute steps in 24 hrs)

for timestep in range(1, T+1):  # Assume T timesteps in a day
    action = agent.act(state)  # Decide action based on current state

    reward, done, next_demand, next_solar, next_wind, next_P_EV = env.step(action)

    # Construct the new state from the separated components
    next_state = np.concatenate([
        np.array([next_demand, next_solar, next_wind]),  # Assuming these are scalar values
        np.array(next_P_EV)  # Assuming this is already an array or list
    ])
    
    agent.remember(state, action, reward, next_state, done)  # Store experience
    #Replay not working?\
    #if len(agent.memory) >= batch_size:
        #agent.replay(batch_size) # Train the agent with a minibatch from memory

    state = next_state  # Move to the next state
    total_reward += reward
    if done:
        break

print("Total reward for the episode (day):", total_reward)


IndexError: index 96 is out of bounds for axis 1 with size 96